In [2]:
import basedosdados as bd
import pandas as pd
from paths import manipulated_data_path, project_name
import os

In [ ]:
query = """
          SELECT ano AS ANO, 
                 mes AS MES,  
                 id_municipio AS COD_MUN,
                 SUM(CASE WHEN admitidos_desligados = '01' THEN 1 ELSE 0 END) AS ADMITIDOS,
                 SUM(CASE WHEN admitidos_desligados = '02' THEN 1 ELSE 0 END) AS DESLIGADOS,
                 SUM(CASE WHEN admitidos_desligados = '01' THEN 1 WHEN admitidos_desligados = '02' THEN -1 ELSE 0 END) AS SALDO

         FROM `basedosdados.br_me_caged.microdados_antigos`

         WHERE ano >= 2002
         
         GROUP BY ano, mes, id_municipio
         
         ORDER BY ano, mes, id_municipio
         """

In [29]:
caged_old_df = bd.read_sql(query=query,
                    billing_project_id=project_name)

Downloading: 100%|██████████|


In [ ]:
caged_old_df.set_index(['ANO', 'MES', 'COD_MUN'], inplace=True)

In [36]:
caged_old_df.head()

ADMITIDOS  DESLIGADOS  SALDO
ANO  MES COD_MUN                              
2007 1   3100104         23          19      4
         3100203        105         123    -18
         3100302         19          20     -1
         3100401          0           5     -5
         3100500          5          10     -5

In [ ]:
query = """
          SELECT CAST(SUBSTR(competencia_movimentacao, 1, 4) AS INT) AS ANO,
                 CAST(SUBSTR(competencia_movimentacao, 6, 2) AS INT) AS MES,
                 id_municipio AS COD_MUN,
                 SUM(CASE WHEN admitidos_desligados = '01' THEN 1 ELSE 0 END) AS ADMITIDOS,
                 SUM(CASE WHEN admitidos_desligados = '02' THEN 1 ELSE 0 END) AS DESLIGADOS,
                 SUM(CASE WHEN admitidos_desligados = '01' THEN 1 WHEN admitidos_desligados = '02' THEN -1 ELSE 0 END) AS SALDO,

         FROM `basedosdados.br_me_caged.microdados_antigos_ajustes`

         WHERE ano >= 2002
         
         GROUP BY ano, mes, id_municipio
         
         ORDER BY ano, mes, id_municipio
         """

In [91]:
caged_old_ajustes_df = bd.read_sql(query=query,
                    billing_project_id=project_name)

Downloading: 100%|██████████|


In [105]:
caged_old_ajustes_df = caged_old_ajustes_df[caged_old_ajustes_df['ANO'] >= 2007]

In [ ]:
caged_old_ajustes_df.set_index(['ANO', 'MES', 'COD_MUN'], inplace=True)

In [110]:
caged_old_ajustes_df.head()

ADMITIDOS  DESLIGADOS  SALDO
ANO  MES COD_MUN                              
2007 1   3100203          0           5     -5
         3100401         13           7      6
         3100500          0           1     -1
         3100906          1           0      1
         3101102          0           2     -2

In [111]:
anos = list(set(list(zip(*list(caged_old_ajustes_df.index)))[0]))
anos.sort()
print(anos)

[2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]


In [112]:
caged_old_joined_df = caged_old_df.add(caged_old_ajustes_df, fill_value=0)

In [113]:
caged_old_joined_df.head()

ADMITIDOS  DESLIGADOS  SALDO
ANO  MES COD_MUN                              
2007 1   3100104         23          19      4
         3100203        105         128    -23
         3100302         19          20     -1
         3100401         13          12      1
         3100500          5          11     -6

In [115]:
anos = list(set(list(zip(*list(caged_old_joined_df.index)))[0]))
anos.sort()
print(anos)

[2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]


In [ ]:
query = """
          SELECT ano AS ANO, 
                 mes AS MES,
                 id_municipio AS COD_MUN,
                 SUM(CASE WHEN saldo_movimentacao = 1 THEN 1 ELSE 0 END) AS ADMITIDOS,
                 SUM(CASE WHEN saldo_movimentacao = -1 THEN 1 ELSE 0 END) AS DESLIGADOS,
                 SUM(saldo_movimentacao) AS SALDO,

         FROM `basedosdados.br_me_caged.microdados_movimentacao`

         WHERE ano >= 2002
         
         GROUP BY ano, mes, id_municipio
         
         ORDER BY ano, mes, id_municipio
         """

In [121]:
caged_new_df = bd.read_sql(query=query,
                    billing_project_id=project_name)

Downloading: 100%|██████████|


In [ ]:
caged_new_df.set_index(['ANO', 'MES', 'COD_MUN'], inplace=True)

In [123]:
caged_new_df.head()

ADMITIDOS  DESLIGADOS  SALDO
ANO  MES COD_MUN                              
2020 1   3100104         48          21     27
         3100203        119         115      4
         3100302         39          63    -24
         3100401          3           2      1
         3100500          6           2      4

In [ ]:
query = """
          SELECT ano_competencia_movimentacao AS ANO, 
                 mes_competencia_movimentacao AS MES,  
                 id_municipio AS COD_MUN,
                 SUM(CASE WHEN saldo_movimentacao = 1 THEN 1 ELSE 0 END) AS ADMITIDOS,
                 SUM(CASE WHEN saldo_movimentacao = -1 THEN 1 ELSE 0 END) AS DESLIGADOS,
                 SUM(saldo_movimentacao) AS SALDO,

         FROM `basedosdados.br_me_caged.microdados_movimentacao_fora_prazo`

         WHERE ano >= 2002
         
         GROUP BY ano, mes, id_municipio
         
         ORDER BY ano, mes, id_municipio
         """

In [125]:
caged_new_ajustes_df = bd.read_sql(query=query,
                    billing_project_id=project_name)

Downloading: 100%|██████████|


In [ ]:
#caged_new_ajustes_df.reset_index(inplace=True)

In [134]:
caged_new_ajustes_df = caged_new_ajustes_df[caged_new_ajustes_df['ANO'] >= 2020]

In [ ]:
caged_new_ajustes_df.set_index(['ANO', 'MES', 'COD_MUN'], inplace=True)

In [138]:
caged_new_ajustes_df.head()

ADMITIDOS  DESLIGADOS  SALDO
ANO  MES COD_MUN                              
2020 1   3100104          3           2      1
         3100203          9           8      1
         3100302          4           6     -2
         3100401          1           0      1
         3100609          1           0      1

In [139]:
caged_new_joined_df = caged_new_df.add(caged_new_ajustes_df, fill_value=0)

In [140]:
anos = list(set(list(zip(*list(caged_new_joined_df.index)))[0]))
anos.sort()
print(anos)

[2020, 2021, 2022, 2023, 2024, 2025]


In [141]:
caged_new_joined_df.head()

ADMITIDOS  DESLIGADOS  SALDO
ANO  MES COD_MUN                              
2020 1   3100104         51          23     28
         3100203        128         123      5
         3100302         43          69    -26
         3100401          4           2      2
         3100500          6           2      4

In [142]:
caged_df = pd.concat([caged_old_joined_df, caged_new_joined_df])

In [143]:
caged_df.head()

ADMITIDOS  DESLIGADOS  SALDO
ANO  MES COD_MUN                              
2007 1   3100104         23          19      4
         3100203        105         128    -23
         3100302         19          20     -1
         3100401         13          12      1
         3100500          5          11     -6

In [144]:
caged_df.to_csv(os.path.join(manipulated_data_path, 'caged_municipios.csv'))

In [ ]:
query = """
          SELECT ano_competencia_movimentacao AS ANO, 
                 mes_competencia_movimentacao AS MES,
                 id_municipio AS COD_MUN,
                 SUM(CASE WHEN saldo_movimentacao = 1 THEN 1 ELSE 0 END) AS ADMITIDOS,
                 SUM(CASE WHEN saldo_movimentacao = -1 THEN 1 ELSE 0 END) AS DESLIGADOS,
                 SUM(saldo_movimentacao) AS SALDO,

         FROM `basedosdados.br_me_caged.microdados_movimentacao_excluida`

         WHERE ano >= 2002
         
         GROUP BY ano, mes, id_municipio
         
         ORDER BY ano, mes, id_municipio
         """

In [146]:
caged_new_excluida_df = bd.read_sql(query=query,
                    billing_project_id=project_name)

Downloading: 100%|██████████|


In [149]:
caged_new_excluida_df = caged_new_excluida_df[caged_new_excluida_df['ANO'] >= 2020]

In [ ]:
caged_new_excluida_df.set_index(['ANO', 'MES', 'COD_MUN'], inplace=True)

In [151]:
caged_new_excluida_df.head()

ADMITIDOS  DESLIGADOS  SALDO
ANO  MES COD_MUN                              
2020 1   3100104          1           0      1
         3100203          0           1     -1
         3101102          0           1     -1
         3101201          3           0      3
         3101508          0           1     -1

In [154]:
teste = caged_new_joined_df.sub(caged_new_excluida_df, fill_value=0)

In [ ]:
#teste.to_csv('teste.csv')